# Работа с росстатом

Данный документ предназначен для обработки и анализа исходных данных росстата

In [0]:
import pandas as pd
import numpy as np
import os
import gc
from tqdm import tqdm

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
os.chdir('/content/drive/Shared drives/Кредитные риски')

In [0]:
from CreditRisks.metrics_library.rosstat_utils import *

## Загрузка датасетов

In [0]:
!wget -O 2016.csv https://www.gks.ru/opendata/7708234640-bdboo2016/data-20191101-structure-20191101.csv

## Поиск различий в ревизиях

Код сравнивает ИНН компаний в датасетах 2018 и 2019 года выпуска

In [0]:
for name in range(YEAR_FIRST, YEAR_LAST):
    df1 = pd.read_csv(f'Датасеты/Росстат/{name}.csv', encoding='windows-1251', names=COLUMNS, header=None, sep=';', index_col=False)
    df2 = pd.read_csv(f'Датасеты/Росстат_2019/{name}.csv', encoding='windows-1251', names=COLUMNS, header=None, sep=';', index_col=False)
    a1 = set(df1['inn'])
    a2 = set(df2['inn'])
    print("Year: ", name)
    print(f"Len 1 {len(a1)}\tLen 2 {len(a2)}")
    print(f"Len & {len(a1 & a2)}\tLen | {len(a1 | a2)}")
    print(f"Len 2 - 1 {len(a2 - a1)}")
    print("\t", a2 - a1)
    print(f"Len 1 - 2 {len(a1 - a2)}")
    print("\t", a1 - a2)
    
    print('=' * 120)
    del df1, df2
    gc.collect()

Year:  2012
Len 1 765768	Len 2 765770
Len & 765768	Len | 765770
Len 2 - 1 2
	 {6670283644, 7804320772}
Len 1 - 2 0
	 set()
Year:  2013
Len 1 1732554	Len 2 1732555
Len & 1732551	Len | 1732558
Len 2 - 1 4
	 {7804320772, 7604012925, 7813501694, 7840445343}
Len 1 - 2 3
	 {9909330080, 5714005187, 5601009085}
Year:  2014
Len 1 1957778	Len 2 1957804
Len & 1957775	Len | 1957807
Len 2 - 1 29
	 {nan, nan, nan, 7804320772.0, 2130060170.0, 7802768015.0, 6315639056.0, 1435223830.0, 6636005911.0, 3015097368.0, 7840445343.0, 7840473125.0, 5609098023.0, 5703005354.0, 7813391018.0, 3812127415.0, 7302021176.0, 7805618748.0, 7801630021.0, 2901216841.0, 7807395663.0, 7801636305.0, 7842405201.0, 7802847066.0, 7810880483.0, 7801637740.0, 7805663853.0, 6509021808.0, 7813501694.0}
Len 1 - 2 3
	 {nan, nan, nan}
Year:  2015
Len 1 2333646	Len 2 2333713
Len & 2333646	Len | 2333713
Len 2 - 1 67
	 {7820044544, 7838501760, 7804320772, 6312073862, 7810418568, 7839322121, 2130060170, 7804274572, 7814630029, 7814201358

## Поиск ошибок в данных (дубликаты, нулевые идентификаторы)

Поиск дубликатов в датасетах, null значений идентификаторов, нормализация представления валюты в 1000 рублей.

Использование ИНН в качестве индекса, сортировка по нему и сохранение результата

In [0]:
for name in range(YEAR_FIRST, YEAR_LAST + 1):
    D = pd.read_csv(f'Датасеты/Росстат_2019/{name}.csv', encoding='windows-1251', names=COLUMNS, header=None, sep=';', index_col=False)
    D.loc[D["measure"] == 383, COLUMNS_VALUE] = round(D[D["measure"] == 383][COLUMNS_VALUE] / 1000).astype('int64')
    D.loc[D["measure"] == 385, COLUMNS_VALUE] = round(D[D["measure"] == 385][COLUMNS_VALUE] * 1000).astype('int64')
    D.drop(columns=["measure", 'name', 'okpo',], inplace=True)

    print("Year: ", name)
    r=D[D['inn'].isna()]
    print(f"Dropping {r.index.shape} na values")
    D.drop(index=r.index, inplace=True)
    ids = D["inn"]
    r = D[ids.isin(ids[ids.duplicated()])]
    print(f"Dropping {r.index.shape} dupl values")
    print(r.index)
    D.drop(index=r.index, inplace=True)
    print(f"INN dtype is {D['inn'].dtype}")
    D['inn'] = D['inn'].astype(np.int64)
    D.set_index('inn', inplace=True)
    D.sort_index(inplace=True)
    
    D.to_csv(f'Датасеты/Росстат_2019_компактный/{name}.csv', index_label='inn')
    print('=' * 120)
    del D, r, ids
    gc.collect()

Year:  2012
Dropping (0,) na values
Dropping (9,) dupl values
Int64Index([452378, 518350, 561226, 570398, 609315, 609316, 609579, 708808,
            708847],
           dtype='int64')
INN dtype is int64
Year:  2013
Dropping (0,) na values
Dropping (10,) dupl values
Int64Index([1190037, 1190044, 1190045, 1190061, 1212438, 1218227, 1218237,
            1218288, 1218366, 1218382],
           dtype='int64')
INN dtype is int64
Year:  2014
Dropping (3,) na values
Dropping (0,) dupl values
Int64Index([], dtype='int64')
INN dtype is float64
Year:  2015
Dropping (0,) na values
Dropping (0,) dupl values
Int64Index([], dtype='int64')
INN dtype is int64
Year:  2016
Dropping (0,) na values
Dropping (0,) dupl values
Int64Index([], dtype='int64')
INN dtype is int64
Year:  2017
Dropping (0,) na values
Dropping (0,) dupl values
Int64Index([], dtype='int64')
INN dtype is int64
Year:  2018
Dropping (0,) na values
Dropping (2,) dupl values
Int64Index([1130055, 1312874], dtype='int64')
INN dtype is int64


## Создание общего списка компаний

In [0]:
companies = {}
COLUMNS_NAME_SAVE = ['name', 'okpo', 'okopf', 'okfs', 'okved', 'inn', ]
for name in range(YEAR_FIRST, YEAR_LAST + 1):
    D = pd.read_csv(f'Датасеты/Росстат_2019/{name}.csv', encoding='windows-1251', names=COLUMNS, header=None, sep=';', index_col=False)

    r=D[D['inn'].isna()]
    D.drop(index=r.index, inplace=True)

    ids = D["inn"]
    r = D[ids.isin(ids[ids.duplicated()])]
    D.drop(index=r.index, inplace=True)

    D['inn'] = D['inn'].astype(np.int64)
    
    companies.update(D[COLUMNS_NAME_SAVE].set_index('inn').to_dict('index'))

    del D, r, ids
    gc.collect()
    print("Done", name , "total", len(companies))

Done 2012 total 765769
Done 2013 total 1875808
Done 2014 total 2394207
Done 2015 total 2981016
Done 2016 total 3372773
Done 2017 total 3689030
Done 2018 total 3933686


In [0]:
df = pd.DataFrame.from_dict(companies, orient='index')
df.sort_index(inplace=True)
df['region'] = df.index // 100000000

In [0]:
df.to_csv(f'Датасеты/Росстат_2019_компактный/companies_info.csv', index_label='inn')